In [1]:
import numpy as np 
import pickle 


In [2]:
grid_option = "10*10"

In [3]:
def load_pickle(name): 
    file = open(name, "rb")
    data = pickle.load(file)
    file.close()
    return data

def read_one_measure(key, path, scale_option =None):
    
    jaco_file = path +key+"_"+scale_option
    jacobian = load_pickle(jaco_file)

    var_file = path + "Var_"+key+"_"+scale_option
    var = load_pickle(var_file)
    
    return jacobian, var

In [4]:
if grid_option == "20*20":
    read_path = "./jaco_and_var/"
    
    measure_len = [40,800,800,800,800,800,800]
    
    num_z_grid, num_t_grid=20,20
    
    total_loc = 800
    
elif grid_option == "10*10":
    read_path = "./jaco_and_var_10t10/"
    
    
    #measure_len = [20,200,200,200,200,200,200]
    measure_len = [20, 200, 200, 180, 200]
    
    num_z_grid, num_t_grid=10,10
    
    total_loc = 200

In [5]:
# from here, we reorganize the measurements order 
# this order matters!!!
measure_keys = ["vg", 
                "Tg", 
                "Ts", 
                "P",
                "y_CO2", 
               # "C_CO2", 
               # "C_N2"
               ]


num_rows = sum(measure_len)

measure_head_row = {}
count = 0 

for i, item in enumerate(measure_keys):
    measure_head_row[item] = count 
    count += measure_len[i]
    
print(measure_head_row)

n_para = 5 



{'vg': 0, 'Tg': 20, 'Ts': 220, 'P': 420, 'y_CO2': 600}


In [6]:
def reorganize(measure_items, scale_option="scale_parameter"):
    
    all_jac = np.zeros((num_rows, n_para))
    all_var = np.zeros((num_rows, 1))
    
    for i, item in enumerate(measure_items):
        
        head_row = measure_head_row[item]
        
        jaco, var = read_one_measure(item, read_path, scale_option = scale_option)
        
        
        if item == "vg": 
            
            # we only records z=1 for adsorption, and z=1 for desorption 
            
            for j in range(num_t_grid):
                row_num = head_row + j
                jaco_head = total_loc//2-num_t_grid
                #print(row_num, jaco_head+j)
                for p in range(n_para):    
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
                    
            for j in range(num_t_grid,2*num_t_grid):
                row_num = head_row + j
                jaco_head = total_loc//2 - num_t_grid
                #print(row_num, jaco_head+j)
                for p in range(n_para):    
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
                    
        elif item=="P":
            for j in range(len(jaco)//2-num_t_grid):
                row_num = head_row+j
                jaco_head = 0 
                print(row_num, jaco_head+j)
                for p in range(n_para):
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
            
            for j in range(len(jaco)//2+num_t_grid, len(jaco)):
                row_num = head_row + j - 2*num_t_grid 
                print(row_num, j)
                for p in range(n_para):
                    all_jac[row_num,p] = jaco[j, p]
                    all_var[row_num] = var[j]
            
        else:
            
            for j in range(len(jaco)):

                row_num = head_row + j 
                #print(row_num)

                for p in range(n_para):
                    all_jac[row_num,p] = jaco[j,p]
                    all_var[row_num] = var[j]
                
    return all_jac, all_var
            

In [7]:
all_jacobian, all_variance = reorganize(measure_keys)
print(measure_head_row)

420 0
421 1
422 2
423 3
424 4
425 5
426 6
427 7
428 8
429 9
430 10
431 11
432 12
433 13
434 14
435 15
436 16
437 17
438 18
439 19
440 20
441 21
442 22
443 23
444 24
445 25
446 26
447 27
448 28
449 29
450 30
451 31
452 32
453 33
454 34
455 35
456 36
457 37
458 38
459 39
460 40
461 41
462 42
463 43
464 44
465 45
466 46
467 47
468 48
469 49
470 50
471 51
472 52
473 53
474 54
475 55
476 56
477 57
478 58
479 59
480 60
481 61
482 62
483 63
484 64
485 65
486 66
487 67
488 68
489 69
490 70
491 71
492 72
493 73
494 74
495 75
496 76
497 77
498 78
499 79
500 80
501 81
502 82
503 83
504 84
505 85
506 86
507 87
508 88
509 89
510 110
511 111
512 112
513 113
514 114
515 115
516 116
517 117
518 118
519 119
520 120
521 121
522 122
523 123
524 124
525 125
526 126
527 127
528 128
529 129
530 130
531 131
532 132
533 133
534 134
535 135
536 136
537 137
538 138
539 139
540 140
541 141
542 142
543 143
544 144
545 145
546 146
547 147
548 148
549 149
550 150
551 151
552 152
553 153
554 154
555 155
556 156
557 

In [8]:
def save_pickle(data, name): 
    file = open(name, "wb")
    pickle.dump(data, file)
    file.close()

save_pickle(all_jacobian, read_path + "jacobian_"+str(num_z_grid)+"_" +str(num_t_grid))

save_pickle(all_variance, read_path + "variance_"+str(num_z_grid)+"_" +str(num_t_grid))


In [26]:
print(all_jacobian[4600:4700])

[]


In [9]:
def compute_fim(overall_jac, overall_var, z_pos_idx=[10,20,30,40]):
    """
    
    """

    var = np.zeros((len(overall_var), len(overall_var)))
    
    for i in range(len(overall_var)):
        var[i,i] = overall_var[i]
        
    var_inv = np.linalg.inv(var)
        
    fim = overall_jac.T@var_inv@overall_jac

    print("===total FIM===")
    print("log(10) determinant:", np.log10(np.linalg.det(fim)))
    print("log(10) trace:", np.log10(np.trace(fim)))
    print("FIM:", fim)

    return fim


In [10]:
compute_fim(all_jacobian, all_variance)

===total FIM===
log(10) determinant: 29.125130036924528
log(10) trace: 10.76903644269845
FIM: [[ 9.82828194e+04 -1.55143473e+04  5.50584189e+06 -1.07659291e+05
   4.87212618e+02]
 [-1.55143473e+04  1.80642678e+05 -7.51614137e+07  1.49142709e+06
  -3.29845675e+02]
 [ 5.50584189e+06 -7.51614137e+07  5.87268704e+10 -9.85950474e+08
   1.81726193e+05]
 [-1.07659291e+05  1.49142709e+06 -9.85950474e+08  2.67158382e+07
   5.08386644e+03]
 [ 4.87212618e+02 -3.29845675e+02  1.81726193e+05  5.08386644e+03
   3.92887156e+01]]


array([[ 9.82828194e+04, -1.55143473e+04,  5.50584189e+06,
        -1.07659291e+05,  4.87212618e+02],
       [-1.55143473e+04,  1.80642678e+05, -7.51614137e+07,
         1.49142709e+06, -3.29845675e+02],
       [ 5.50584189e+06, -7.51614137e+07,  5.87268704e+10,
        -9.85950474e+08,  1.81726193e+05],
       [-1.07659291e+05,  1.49142709e+06, -9.85950474e+08,
         2.67158382e+07,  5.08386644e+03],
       [ 4.87212618e+02, -3.29845675e+02,  1.81726193e+05,
         5.08386644e+03,  3.92887156e+01]])